In [25]:
import pandas as pd
import json
import numpy
from pandas.io.json import json_normalize
from scipy.spatial.distance import cosine
import os.path
import sys
sys.path.append('../python/')
import cdr
import imp
imp.reload(cdr)

<module 'cdr' from '../python/cdr.py'>

In [2]:
df = pd.read_csv('../../data/CDR/sms-call-internet-mi-2013-11-01.txt', delimiter='\t',
                 header=None) 

In [3]:
file = '../../data/GeoJSON/milano-grid.geojson'
with open(file) as f:
    grid = pd.read_json(f, typ='Series')

In [5]:
df.columns = ["cellId", "time", "countryCode", "smsIn", "smsOut", "callIn", "callOut", "internet"]
norm_grid = json_normalize(grid['features'])

In [6]:
norm_grid.head()

,geometry.coordinates,geometry.type,id,properties.cellId,type
0,"[[[9.0114910478323, 45.35880131440966], [9.014...",Polygon,0,1,Feature
1,"[[[9.014491488013135, 45.35880097314403], [9.0...",Polygon,1,2,Feature
2,"[[[9.017491928134044, 45.358800553060284], [9....",Polygon,2,3,Feature
3,"[[[9.02049236818262, 45.35880005415845], [9.02...",Polygon,3,4,Feature
4,"[[[9.023492808146456, 45.35879947643852], [9.0...",Polygon,4,5,Feature


In [27]:
df.index = pd.to_datetime(df['time'],unit='ms',utc=True)

In [29]:
df.index = df.index.tz_localize('UTC').tz_convert('Europe/Rome')

In [59]:
df.head()

,cellId,time,countryCode,smsIn,smsOut,callIn,callOut,internet,date,time_hour,weekday,morning,morning_weekday,day_weekday,evening_weekday,morning_weekend,day_weekend,evening_weekend
time,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00+01:00,1,1383260400000,0,0.081363,NaN,NaN,NaN,NaN,2013-11-01 00:00:00+01:00,0,4,True,True,False,False,False,False,False
2013-11-01 00:00:00+01:00,1,1383260400000,39,0.141864,0.156787,0.160938,0.052275,11.028366,2013-11-01 00:00:00+01:00,0,4,True,True,False,False,False,False,False
2013-11-01 00:10:00+01:00,1,1383261000000,0,0.136588,NaN,NaN,0.027300,NaN,2013-11-01 00:10:00+01:00,0,4,True,True,False,False,False,False,False
2013-11-01 00:10:00+01:00,1,1383261000000,33,NaN,NaN,NaN,NaN,0.026137,2013-11-01 00:10:00+01:00,0,4,True,True,False,False,False,False,False
2013-11-01 00:10:00+01:00,1,1383261000000,39,0.278452,0.119926,0.188777,0.133637,11.100963,2013-11-01 00:10:00+01:00,0,4,True,True,False,False,False,False,False


In [31]:
df['date'] = df.index

In [51]:
df['time_hour'] = df.index.hour

In [48]:
df['weekday'] = df.index.weekday

In [57]:
# returning Boolean
df['morning_weekday'] = (df['time_hour'] >= 0) & (df['time_hour'] < 8) & (df['weekday'] != 6) & (df['weekday'] != 5)
df['day_weekday'] = (df['time_hour'] >= 8) & (df['time_hour'] < 16) & (df['weekday'] != 6) & (df['weekday'] != 5)
df['evening_weekday'] = (df['time_hour'] >= 16) & (df['time_hour'] < 24) & (df['weekday'] != 6) & (df['weekday'] != 5)
df['morning_weekend'] = (df['time_hour'] >= 0) & (df['time_hour'] < 8) & ((df['weekday'] == 6) | (df['weekday'] == 5))
df['day_weekend'] = (df['time_hour'] >= 8) & (df['time_hour'] < 16) & ((df['weekday'] == 6) | (df['weekday'] == 5))
df['evening_weekend'] = (df['time_hour'] >= 16) & (df['time_hour'] < 24) & ((df['weekday'] == 6) | (df['weekday'] == 5))


In [71]:
morning_weekday.head()

,time,callIn,internet,callOut,cellId,smsIn,smsOut
cellId,,,,,,,
1,1383260400000,1.305744,320.643279,1.510645,1,5.450417,4.137999
2,1383260400000,1.323735,321.442663,1.539763,2,5.477261,4.145378
3,1383260400000,1.342885,322.293575,1.570757,3,5.505834,4.153233
4,1383260400000,1.253634,318.327848,1.426305,4,5.372665,4.116626
5,1383260400000,1.174183,290.004421,1.326145,5,4.869727,3.739232


In [68]:
morning_weekday = df[(df['countryCode'] != 0) & (df['morning_weekday'] == True)].groupby('cellId').agg({
                    'cellId': 'first',
                    'time': 'first',
                    'smsIn': 'sum',
                    'smsOut': 'sum',
                    'callIn': 'sum',
                    'callOut': 'sum',
                    'internet': 'sum'
                })

In [69]:
day_weekday = df[(df['countryCode'] != 0) & (df['day_weekday'] == True)].groupby('cellId').agg({
                    'cellId': 'first',
                    'time': 'first',
                    'smsIn': 'sum',
                    'smsOut': 'sum',
                    'callIn': 'sum',
                    'callOut': 'sum',
                    'internet': 'sum'
                })
evening_weekday = df[(df['countryCode'] != 0) & (df['evening_weekday'] == True)].groupby('cellId').agg({
                    'cellId': 'first',
                    'time': 'first',
                    'smsIn': 'sum',
                    'smsOut': 'sum',
                    'callIn': 'sum',
                    'callOut': 'sum',
                    'internet': 'sum'
                })
morning_weekend = df[(df['countryCode'] != 0) & (df['morning_weekend'] == True)].groupby('cellId').agg({
                    'cellId': 'first',
                    'time': 'first',
                    'smsIn': 'sum',
                    'smsOut': 'sum',
                    'callIn': 'sum',
                    'callOut': 'sum',
                    'internet': 'sum'
                })
day_weekend = df[(df['countryCode'] != 0) & (df['day_weekend'] == True)].groupby('cellId').agg({
                    'cellId': 'first',
                    'time': 'first',
                    'smsIn': 'sum',
                    'smsOut': 'sum',
                    'callIn': 'sum',
                    'callOut': 'sum',
                    'internet': 'sum'
                })
evening_weekend = df[(df['countryCode'] != 0) & (df['evening_weekend'] == True)].groupby('cellId').agg({
                    'cellId': 'first',
                    'time': 'first',
                    'smsIn': 'sum',
                    'smsOut': 'sum',
                    'callIn': 'sum',
                    'callOut': 'sum',
                    'internet': 'sum'
                })

In [74]:
m_weekday = pd.merge(left=norm_grid, right=morning_weekday, how='left', left_on='properties.cellId', right_on='cellId')
d_weekday = pd.merge(left=norm_grid, right=day_weekday, how='left', left_on='properties.cellId', right_on='cellId')
e_weekday = pd.merge(left=norm_grid, right=evening_weekday, how='left', left_on='properties.cellId', right_on='cellId')
m_weekend = pd.merge(left=norm_grid, right=morning_weekend, how='left', left_on='properties.cellId', right_on='cellId')
d_weekend = pd.merge(left=norm_grid, right=day_weekend, how='left', left_on='properties.cellId', right_on='cellId')
e_weekend = pd.merge(left=norm_grid, right=evening_weekend, how='left', left_on='properties.cellId', right_on='cellId')

In [75]:
m_weekday.head()

,geometry.coordinates,geometry.type,id,properties.cellId,type,time,callIn,internet,callOut,cellId,smsIn,smsOut
0,"[[[9.0114910478323, 45.35880131440966], [9.014...",Polygon,0,1,Feature,1383260400000,1.305744,320.643279,1.510645,1,5.450417,4.137999
1,"[[[9.014491488013135, 45.35880097314403], [9.0...",Polygon,1,2,Feature,1383260400000,1.323735,321.442663,1.539763,2,5.477261,4.145378
2,"[[[9.017491928134044, 45.358800553060284], [9....",Polygon,2,3,Feature,1383260400000,1.342885,322.293575,1.570757,3,5.505834,4.153233
3,"[[[9.02049236818262, 45.35880005415845], [9.02...",Polygon,3,4,Feature,1383260400000,1.253634,318.327848,1.426305,4,5.372665,4.116626
4,"[[[9.023492808146456, 45.35879947643852], [9.0...",Polygon,4,5,Feature,1383260400000,1.174183,290.004421,1.326145,5,4.869727,3.739232


In [77]:
cols = ['cellId','smsIn','smsOut','callIn','callOut','internet']
m_wd = cdr.df_to_geojson(m_weekday, cols)
d_wd = cdr.df_to_geojson(d_weekday, cols)
e_wd = cdr.df_to_geojson(e_weekday, cols)
m_we = cdr.df_to_geojson(m_weekend, cols)
d_we = cdr.df_to_geojson(d_weekend, cols)
e_we = cdr.df_to_geojson(e_weekend, cols)

In [80]:
# writing the file
output_filename = '../../data/CDR/generated/CDR_morning_weekday.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(m_wd, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_day_weekday.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(d_wd, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_evening_weekday.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(e_wd, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_morning_weekend.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(m_we, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_day_weekend.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(d_we, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_evening_weekend.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(e_we, output_file, indent=2)

print ("file generated.")

file generated.


In [10]:
dfs = {}

# Read all the files in Milano
# for month in {"11", "12"}:
#     for day in range(1, 32):
for month in {"11"}:
    for day in range(1, 3):
        to_read = '../../data/CDR/sms-call-internet-mi-2013-' + month + '-' +\
                    str(day).zfill(2) + '.txt'
        
        if os.path.isfile(to_read):
            dfs[month + str(day).zfill(2)] = pd.read_csv(to_read, delimiter='\t', header=None) 
            print ("loaded ", to_read)

loaded  ../../data/CDR/sms-call-internet-mi-2013-11-01.txt
loaded  ../../data/CDR/sms-call-internet-mi-2013-11-02.txt


In [11]:
print ("Milano files loaded")

file = '../../data/GeoJSON/milano-grid.geojson'
with open(file) as f:
    grid = pd.read_json(f, typ='Series')

print ("Grid file loaded")

Milano files loaded
Grid file loaded


In [23]:
abc = dfs['1101'].copy()

In [24]:
abc.head()


,0,1,2,3,4,5,6,7
0,1,1383260400000,0,0.081363,NaN,NaN,NaN,NaN
1,1,1383260400000,39,0.141864,0.156787,0.160938,0.052275,11.028366
2,1,1383261000000,0,0.136588,NaN,NaN,0.027300,NaN
3,1,1383261000000,33,NaN,NaN,NaN,NaN,0.026137
4,1,1383261000000,39,0.278452,0.119926,0.188777,0.133637,11.100963


In [26]:
m_weekd, d_weekd, e_weekd, m_weeke, d_weeke, e_weeke = cdr.join_cdr_grid_by_time(dfs['1101'], grid)

In [12]:
dfs['1101'].head()

,0,1,2,3,4,5,6,7
0,1,1383260400000,0,0.081363,NaN,NaN,NaN,NaN
1,1,1383260400000,39,0.141864,0.156787,0.160938,0.052275,11.028366
2,1,1383261000000,0,0.136588,NaN,NaN,0.027300,NaN
3,1,1383261000000,33,NaN,NaN,NaN,NaN,0.026137
4,1,1383261000000,39,0.278452,0.119926,0.188777,0.133637,11.100963


In [9]:
dfs

{'1101':                            cellId           time  countryCode     smsIn  \
 time                                                                      
 2013-11-01 00:00:00+01:00       1  1383260400000            0  0.081363   
 2013-11-01 00:00:00+01:00       1  1383260400000           39  0.141864   
 2013-11-01 00:10:00+01:00       1  1383261000000            0  0.136588   
 2013-11-01 00:10:00+01:00       1  1383261000000           33       NaN   
 2013-11-01 00:10:00+01:00       1  1383261000000           39  0.278452   
 2013-11-01 00:20:00+01:00       1  1383261600000            0  0.053438   
 2013-11-01 00:20:00+01:00       1  1383261600000           39  0.330641   
 2013-11-01 00:30:00+01:00       1  1383262200000            0  0.026137   
 2013-11-01 00:30:00+01:00       1  1383262200000           39  0.681434   
 2013-11-01 00:40:00+01:00       1  1383262800000            0  0.027300   
 2013-11-01 00:40:00+01:00       1  1383262800000           39  0.243378   
 201